In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset_2019_2022.csv')

In [3]:
df.head(10)

,customer_id,product_id,basket_id,loyalty,household_type,age_band,department,brand,commodity,store,price,transaction_date
0,15803,1131974,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Baked bread/buns/rolls,374,0.99,5/10/2020
1,15803,1051516,57266,Loyalist,1 adult with kids,19-24,Produce,national,Vegetables - all others,374,0.70,24/10/2020
2,15803,967254,57266,Loyalist,1 adult with kids,19-24,Pharmaceutical,national,Cold and flu,374,1.68,18/10/2020
3,15803,1134222,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Paper housewares,374,2.59,23/10/2020
4,15803,1003421,57266,Loyalist,1 adult with kids,19-24,Grocery,national,Soup,374,0.60,27/10/2020
5,15803,937791,57266,Loyalist,1 adult with kids,19-24,Grocery,national,Frozen pizza,374,2.25,28/10/2020
6,15803,1082185,57266,Loyalist,1 adult with kids,19-24,Produce,national,Tropical fruit,374,1.07,12/10/2020
7,15803,940947,57266,Loyalist,1 adult with kids,19-24,Meat,national,Heat/serve,374,5.00,2/10/2020
8,15803,1056010,57266,Loyalist,1 adult with kids,19-24,Nutrition,national,Prepared food,374,3.00,17/10/2020
9,15803,845307,57266,Loyalist,1 adult with kids,19-24,Deli,national,Deli meats,374,3.17,24/10/2020


In [4]:
df_clean = df.copy()

In [5]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77750 entries, 0 to 77749
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       77750 non-null  int64  
 1   product_id        77750 non-null  int64  
 2   basket_id         77750 non-null  int64  
 3   loyalty           77750 non-null  object 
 4   household_type    77750 non-null  object 
 5   age_band          77750 non-null  object 
 6   department        77750 non-null  object 
 7   brand             77750 non-null  object 
 8   commodity         77750 non-null  object 
 9   store             77750 non-null  int64  
 10  price             77750 non-null  float64
 11  transaction_date  77750 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 7.1+ MB


In [6]:
#using loyalty column and replacing Promiscious with Non-Loyalist
df_clean['loyalty'] = df_clean['loyalty'].replace("Promiscuous", "Non-Loyalist")
np.unique(df_clean["loyalty"])

array(['First Time Buyer', 'Loyalist', 'Non-Loyalist'], dtype=object)

In [7]:
df_clean.shape

(77750, 12)

In [8]:
#Remove duplicates
df_clean.drop_duplicates(keep= 'first', inplace=True)
df_clean.shape

(77691, 12)

In [9]:
#Check for null values within dataframe
df_clean.isnull().sum()

customer_id         0
product_id          0
basket_id           0
loyalty             0
household_type      0
age_band            0
department          0
brand               0
commodity           0
store               0
price               0
transaction_date    0
dtype: int64

In [10]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77691 entries, 0 to 77749
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       77691 non-null  int64  
 1   product_id        77691 non-null  int64  
 2   basket_id         77691 non-null  int64  
 3   loyalty           77691 non-null  object 
 4   household_type    77691 non-null  object 
 5   age_band          77691 non-null  object 
 6   department        77691 non-null  object 
 7   brand             77691 non-null  object 
 8   commodity         77691 non-null  object 
 9   store             77691 non-null  int64  
 10  price             77691 non-null  float64
 11  transaction_date  77691 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 7.7+ MB


In [11]:
#Filter out customer_ids longer than 5 digits 
df_clean = df_clean[df_clean['customer_id'].astype(str).str.len() == 5]

In [12]:
df_clean.shape

(77672, 12)

In [13]:
(df_clean.groupby('basket_id')['transaction_date'].nunique() > 1).sum()

3479

In [14]:
#Standardise dates in the dataframe to transform dates to the first day of the month and remove time references. 
#this also negates some anomalies in the data where the same basket_id is found with multiple dates
df_clean['t_date'] = pd.to_datetime(df_clean['transaction_date'], format="%d/%m/%Y")  # Convert to datetime
df_clean['t_date'] = df_clean['t_date'].dt.to_period('M').dt.start_time

In [15]:
df_clean.groupby('basket_id').agg(trans_count=('t_date', 'nunique')).head(10)

,trans_count
basket_id,
51964,1
51965,1
51966,1
51967,1
51968,1
51969,1
51970,1
51971,1
51972,1


In [16]:
#Confirming all basket_id dates have been standardised
(df_clean.groupby('basket_id')['t_date'].nunique() > 1).sum()

0

In [17]:
df_clean.to_csv('cleaned_SFM_dataset_2019_2022.csv', index=False)